In [5]:
import requests
import json
import geojson
import ipywidgets as widgets

import pandas as pd
import xarray as xr

# Download some data
First we need to get some data. In this example we are interested in the production of salmon and trout in Norway. The Norwegian Fisheries Directorate provides this data in the form of xlsx files. But when we bring that into our environment, it needs some work to clean it up.

In [2]:
url = "https://www.fiskeridir.no/English/Aquaculture/Statistics/Atlantic-salmon-and-rainbow-trout/grow-out-production/sta-laks-mat-11-beh-bevegelse.xlsx"

# Read all sheets into a dictionary of DataFrames
xlsx_data = pd.read_excel(url, sheet_name=None)

# Concatenate all DataFrames into one with a new 'Year' column
combined_dfs = []
for year, df in xlsx_data.items():
    df['Year'] = year  # Add a new column with the year
    combined_dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(combined_dfs, ignore_index=True)

# Print or further process the combined DataFrame
print(combined_df)

# Transforming up the data
In the next few cells, we are going to be transforming the dataset into something that is easier to work with. By the end of this process, we will have a nice table with the production data by year for both trout and salmon.

In [3]:
# Assuming you have already executed the code to create the 'combined_df' DataFrame
display(combined_df)


In [4]:
# Filtering the data
# Specify the desired terms
desired_terms = [
    "Troms og Finnmark",
    "Nordland",
    "Trøndelag",
    "Møre og Romsdal",
    "Vestland",
    "Rogaland",
    "Øvrige fylker"
]

# Filter the DataFrame based on the first column
filtered_df = combined_df[combined_df.iloc[:, 0].isin(desired_terms)]

# Duplicate the rows
filtered_df = pd.concat([filtered_df] * 2, ignore_index=True)

# Add a new column labeling rows with "Rainbow Trout" or "Atlantic Salmon"
filtered_df['Label'] = ['Rainbow Trout'] * (len(filtered_df) // 2) + ['Atlantic Salmon'] * (len(filtered_df) // 2)

# Display the modified DataFrame
display(filtered_df)

In [5]:
# Define the mapping for column renaming
column_mapping = {
    'Laks, regnbueørret og ørret - matfiskproduksjon': 'County',
    'Unnamed: 1': 'Live stock pr. 1.1',
    'Unnamed: 2': 'Input',
    'Unnamed: 3': 'Output',
    'Unnamed: 4': 'Losses',
    'Unnamed: 5': 'Live stock pr. 12.31.',
    'Label': 'Species'
}

# Rename columns in the DataFrame
filtered_df.rename(columns=column_mapping, inplace=True)

# Display the DataFrame with the new column names
display(filtered_df)

In [6]:
df = pd.DataFrame(filtered_df)

# Set hierarchical index with 'County', 'Year', and 'Species'
df.set_index(['County', 'Year', 'Species'], inplace=True)

# Sort the index for better readability
df.sort_index(axis=0, level=['County', 'Year', 'Species'], inplace=True)

# Reset the index to make 'County', 'Year', and 'Species' regular columns
df.reset_index(inplace=True)

# Display the modified DataFrame
display(df)

In [7]:
# Duplicate the rows with the first three columns
filtered_df = pd.concat([df.iloc[:, :3]] * 2, ignore_index=True)

# Add the last five columns' values under columns 4 to 8
filtered_df[df.columns[3:8]] = df.iloc[:, 3:].values.reshape(-1, 5)

# Display the modified DataFrame
display(filtered_df)

# Exporting as csv
Now we have a nice looking table, we can export it as a csv file or take it one step further and upload it to our own data collection in ODP.

In [8]:
# Save pandas to csv
csv_filename = "atlantic-salmon-rainbow-trout-and-trout-grow-out-production.csv"

# Save the DataFrame to a CSV file
filtered_df.to_csv(csv_filename, index=False)

# Display a message indicating successful save
print(f"DataFrame has been saved to {csv_filename}")

# Uploading Data to ODP
In the next step, we are going to be doing a few things that will allow us to upload our data to ODP.
1. Creating a data collection
2. Creating a dataset inside collection
3. Definin a schema for our data
4. Uploading the data to ODP

We are going to be following the quickstart guide to help us: https://docs.hubocean.earth/guides/quickstart/
    
But first we need our token.

In [4]:
token = requests.post("http://localhost:8000/access_token").json()['token']

In [1]:
print(token)

In [10]:
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "X-ODP-CHUNKED-ENCODING": "false"
}
base_url = "https://api.hubocean.earth"



## Creating a Data Collection
First we need to create our data collection. In the first cell we name both the collection and the dataset. These names need to be computer friendly, but we can make nicer ones later on.

In [11]:
dataset_collection_name = "norwegian-directorat-of-fisheries-grow-out-production" # Use the existing name or make-your-own-computer-friendly-name
dataset_name = "overview-of-the-live-stock-1998-2022" # Use the existing name or make-your-own-computer-friendly-name

In [12]:
# Create data collection

endpoint = f"/catalog"
url = base_url + endpoint

body = {
    "kind": "catalog.hubocean.io/dataCollection",
    "version": "v1alpha1",
    "metadata": {
        "name": f"{dataset_collection_name}",
        "display_name": "Norwegian Directorat of Fisheries- Grow Out Production",
        "description": "Overview of the livestock 1998-2022",
        "labels": {
            "hubocean.io/test": 'true'
        }
    },
    "spec": {
        "distribution": {
            "published_by": {
                "contact": "Tom Redd <mail@address.earth>",
                "organisation": "HUB Ocean"
            },
            "published_date": "2019-06-19T06:00:00",
            "website": "https://hubocean.earth",
            "license": {
                "name": "propriatary",
                "full_text": "This is a very strict legal text describing the data license for this data collection. The lawyer would be proud.",
                "href": "www.license.com"
            }
        },
        "tags": ["test", "hubocean"]
    }
}

response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    json_response = response.json()
else:
       print(f"Request failed with status code {response.status_code} - {response.text}")

In [13]:
## Check to see if the data collection exists

resource_group = "catalog.hubocean.io"
resource_type = "dataCollection"
endpoint = f"/catalog/{resource_group}/{resource_type}/{dataset_collection_name}"
url = base_url + endpoint

response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_response = response.json()
else:
   print(f"Request failed with status code {response.status_code} - {response.text}")

In [14]:
json_response

## Creating a Dataset
Now we have a data collection we can make one or multiple datasets inside it.

In [15]:
#dataset inside collection

endpoint = "/catalog"
url = base_url + endpoint

body = {
    "kind": "catalog.hubocean.io/dataset",
    "version": "v1alpha3",
    "metadata": {
        "name": f"{dataset_name}",
        "display_name": "Atlantic salmon, Rainbow trout and Trout - Grow out production",
        "description": "Official statistics",
        "labels": {
            "hubocean.io/test": "true"
        }
    },
    "spec": {
        "data_collection": f"catalog.hubocean.io/dataCollection/{dataset_collection_name}",
        "storage_class": "registry.hubocean.io/storageClass/tabular",
        "storage_controller": "registry.hubocean.io/storageController/storage-tabular",
        "maintainer": {
            "contact": "Redd, Tom <tom.redd@oceandata.earth>",
            "organisation": "HUB Ocean"
        }
    }
}

response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    json_response = response.json()
    print(json_response)

else:
    print(f"Request failed with status code {response.status_code} - {response.text}")

## Creating a Schema
Before we upload the data, we need to make a schema describing the data. If you look above you will see that the schema coresponds to the column headings.

In [16]:
# Create table schema

kind = "catalog.hubocean.io/dataset"

endpoint = f"/data/{kind}/{dataset_name}/schema"
body = {
    "table_schema": {
        "County": {
            "type": "string"
        },
        "Year": {
            "type": "string"
        },
        "Species": {
            "type": "string"
        },
        "Live stock pr. 1.1": {
            "type": "double"
        },
        "Input": {
            "type": "double"
        },
        "Output": {
            "type": "double"
        },
        "Losses": {
            "type": "double"
        },
        "Live stock pr. 12.31.": {
            "type": "double"
        },
    },
    "table_description": "Overview over the live stock by county. The number of units in 1000",
    "geospatial_partition_columns": [
        "Location"
    ],
    "geospatial_partition_hash_precision": 5,
    "table_metadata": {
        "geometry": {
            "primary_location": "Location"
        }
    }
}


url = base_url + endpoint
response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    json_response = response.json()
    print(json_response)

else:
    print(f"Request failed with status code {response.status_code} - {response.text}")

In [17]:
# Get Schema by dataset name

kind = "catalog.hubocean.io/dataset"

endpoint = f"/data/{kind}/{dataset_name}/schema"

url = base_url + endpoint
response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_response = response.json()
    print(json_response)

else:
    print(f"Request failed with status code {response.status_code} - {response.text}")

## Uploading Data to ODP
Now we actually get to upload the data from the table we created. We are using a simple function to automate this process

In [18]:
# Create datapoints for all rows in the DataFrame
datapoints = []

for index, row in filtered_df.iterrows():
    datapoint = {
        "County": row['County'],
        "Year": row['Year'],
        "Species": row['Species'],
        "Live stock pr. 1.1": row['Live stock pr. 1.1'],
        "Input": row['Input'],
        "Output": row['Output'],
        "Losses": row['Losses'],
        "Live stock pr. 12.31.": row['Live stock pr. 12.31.'],
    }
    datapoints.append(datapoint)

# Create the body for the request
body = {"data": datapoints}
print(datapoints[1])


In [19]:
#

kind = "catalog.hubocean.io/dataset"

endpoint = f"/data/{kind}/{dataset_name}"

url = base_url + endpoint

response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    json_response = response.json()
    print(json_response)
else:
    print(f"Request failed with status code {response.status_code} - {response.text}")

## Not sure what we are doing here...

In [20]:
# Query for our dataset with the OQS syntax.

endpoint = "/catalog/list"
body = {
    "oqs": {
        "#EQUALS": [
            "$metadata.name",
            "overview-of-the-live-stock-1998-2022"
        ]
    }
}



url = base_url + endpoint
response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    json_response = response.json()
    print(json_response)

else:
    print(f"Request failed with status code {response.status_code} - {response.text}")


# Pulling Data from the ODP API
Now the data is stored in ODP, we can test to see if its working and start to think about how we might analyse it. To make things a bit more managable, you might want to filter the data by county and year so lets make a something to help with that. We then use this input to filter the data we pull from the API.

In [21]:
# Define the list of counties
county_list = [
    'Troms og Finnmark',
    'Nordland',
    'Trøndelag',
    'Møre og Romsdal',
    'Vestland',
    'Rogaland',
    'Øvrige fylker'
]

# Define a dropdown widget for selecting the county
county_dropdown = widgets.Dropdown(
    options=county_list,
    value=county_list[0],  # Set the default selected county
    description='Select County:',
    disabled=False,
)

# Create a slider widget
year_slider = widgets.IntSlider(
    value=2022,  # Initial value
    min=1998,    # Minimum year
    max=2030,    # Maximum year
    step=1,      # Step size
    description='Select Year:',
    continuous_update=False  # Set to False to update only on slider release
)

# Display the slider
display(year_slider)

# Display the dropdown widget
display(county_dropdown)

In [22]:
selected_county = county_dropdown.value
selected_year = year_slider.value
print(selected_county)
print(selected_year)

In [3]:
# Query for our dataset with the OQS syntax.

resource_group = "catalog.hubocean.io"
resource_type = "dataset"

endpoint = f"/data/{resource_group}/{resource_type}/{dataset_name}/list"


body = {
    "filters": {
        "#AND": [
            {
                "#EQUALS": ["$County", selected_county]
            },
            {
                "#EQUALS": ["$Year", selected_year]
            }
            # Add more filters if needed
        ]
    }
}



url = base_url + endpoint
response = requests.post(url, json=body, headers=headers)

if response.status_code == 200:
    print(response.json())

else:
    print(f"Request failed with status code {response.status_code} - {response.text}")
    
pd.json_normalize(data_json_response['data'], max_level=0)